In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from subprocess import check_output
print(check_output(["ls", "input"]).decode("utf8"))


train (1).csv



In [4]:
import keras
from keras.models import Sequential
from keras.layers import Dense,Activation,Conv2D,MaxPool2D,Flatten,Dropout
from keras.utils import np_utils
from sklearn.cross_validation import train_test_split

Using TensorFlow backend.
/home/rfdickerson/.local/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [6]:
df_train = pd.read_csv("input/train.csv")
df_train.head(3)

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
features = ["%s%s" %("pixel",pixel_no) for pixel_no in range(0,784)]
df_train_features = df_train[features]
df_train_features.shape

(42000, 784)

In [9]:
df_train_labels = df_train["label"]
df_train_labels_categorical = np_utils.to_categorical(df_train_labels)
df_train_labels_categorical[0:3]

array([[ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [10]:
X_train,X_test,y_train,y_test = train_test_split(df_train_features, df_train_labels_categorical, test_size=0.10,random_state=32)

## Architecture of Keras Model

In [11]:
model = Sequential()
model.add(Dense(32,activation='relu',input_dim=784))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.03))
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.03))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(10, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adadelta(), metrics=['accuracy'])

model.fit(X_train.values, y_train, batch_size=128, epochs=50, verbose=1)

Epoch 1/50
37800/37800 [==============================] - 2s - loss: 2.3599 - acc: 0.5749     
Epoch 2/50
37800/37800 [==============================] - 1s - loss: 0.6720 - acc: 0.8051     
Epoch 3/50
37800/37800 [==============================] - 1s - loss: 0.4511 - acc: 0.8711     
Epoch 4/50
37800/37800 [==============================] - 1s - loss: 0.3566 - acc: 0.8985     
Epoch 5/50
37800/37800 [==============================] - 1s - loss: 0.3023 - acc: 0.9149     
Epoch 6/50
37800/37800 [==============================] - 1s - loss: 0.2718 - acc: 0.9222     
Epoch 7/50
37800/37800 [==============================] - 1s - loss: 0.2425 - acc: 0.9289     
Epoch 8/50
37800/37800 [==============================] - 1s - loss: 0.2224 - acc: 0.9360     
Epoch 9/50
37800/37800 [==============================] - 1s - loss: 0.2069 - acc: 0.9401     
Epoch 10/50
37800/37800 [==============================] - 1s - loss: 0.1943 - acc: 0.9435     
Epoch 11/50
37800/37800 [========================

## Test

In [14]:
df_test = pd.read_csv("input/test.csv")
df_test.head(3)

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
pred_classes = model.predict_classes(df_test.values)

27328/28000 [============================>.] - ETA: 0s

In [16]:
submission = pd.DataFrame({
    "ImageId": df_test.index+1,
    "Label": pred_classes
})
print(submission[0:10])

submission.to_csv('./keras_model_1.csv', index=False)

   ImageId  Label
0        1      2
1        2      0
2        3      9
3        4      9
4        5      3
5        6      7
6        7      0
7        8      3
8        9      0
9       10      3
